## Prerequisites

In order to run this notebook one needs to have pyclesperanto installed !

For details on how to do this check: [pyclesperanto_prototype](https://github.com/clEsperanto/pyclesperanto_prototype)

In [23]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [24]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ipyfilechooser
  ! pip install pyclesperanto-prototype
  ! pip install pyopencl

In [25]:
# import the required libraries
from czitools import metadata_tools as czimd
from czitools import misc_tools
from pylibCZIrw import czi as pyczi
from pathlib import Path
import os
import pandas as pd
from skimage import measure, segmentation
from tqdm.contrib.itertools import product
import pyclesperanto_prototype as cle
from typing import List, Dict, Tuple, Optional, Type, Any, Union
import numpy as np
import requests

In [26]:
def cleseg_voroni_otsu(image: np.ndarray,
                    sigma_spot_detection: int = 5,
                    sigma_outline: int = 1,
                    convert2numpy: bool = True,
                    verbose: bool = False) -> Union[np.ndarray, cle.Image]:

    # based on: https://biapol.github.io/HIP_Introduction_to_Napari_and_image_processing_with_Python_2022/07_Image_segmentation/02_voronoi_otsu_labeling.html

    # transfer the image to the GPU
    image_to_segment = cle.asarray(image)

    # blur the image with a given sigma and detect maxima in the resulting image.
    blurred = cle.gaussian_blur(image_to_segment, sigma_x=sigma_spot_detection, sigma_y=sigma_spot_detection, sigma_z=sigma_spot_detection)

    detected_spots = cle.detect_maxima_box(blurred, radius_x=0, radius_y=0, radius_z=0)

    if verbose:
        number_of_spots = cle.sum_of_all_pixels(detected_spots)
        print("Detected spots", number_of_spots)

    # blur it again with a different sigma and run threshold the image
    blurred2 = cle.gaussian_blur(image_to_segment, sigma_x=sigma_outline, sigma_y=sigma_outline, sigma_z=sigma_outline)
    binary = cle.threshold_otsu(blurred2)

    # take the binary spots and segmentation image, apply a binary_and
    # to exclude spots which were detected in the background area.
    selected_spots = cle.binary_and(binary, detected_spots)

    if verbose:
        number_of_spots = cle.sum_of_all_pixels(selected_spots)
        print("Selected spots", number_of_spots)

    # convert back to numpy array
    labeling = cle.masked_voronoi_labeling(selected_spots, binary)

    if convert2numpy:
        labeling = cle.pull(labeling)

    return labeling

In [27]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [28]:
if IN_COLAB:
    filepath = os.path.join(os.getcwd(), "data/w96_A1+A2.czi")

if not IN_COLAB:
    defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
    filepath = os.path.join(defaultdir, "w96_A1+A2.czi")

print("Selected FilePath: ", filepath)

Selected FilePath:  F:\Github\czitools\data\w96_A1+A2.czi


In [29]:
if IN_COLAB:
    # see: https://forum.image.sc/t/stackview-pyclesperanto-prototype-demo-on-colab-cant-use-gpu/80145/4?u=sebi06
    cle.select_device("cupy")

if not IN_COLAB:
    # list names of all available OpenCL-devices
    print("Available OpenCL devices:" + str(cle.available_device_names()))

    # select a specific OpenCL / GPU device and see which one was chosen
    device = cle.select_device("NVIDIA RTX A3000 Laptop GPU")
    print("Used GPU: ", device)

Available OpenCL devices:['NVIDIA RTX A3000 Laptop GPU', 'Intel(R) UHD Graphics']
Used GPU:  <NVIDIA RTX A3000 Laptop GPU on Platform: NVIDIA CUDA (2 refs)>


In [30]:
# define columns names for dataframe for the measure objects
cols = ["WellId", "Well_ColId", "Well_RowId", "S", "T", "Z", "C", "Number"]
objects = pd.DataFrame(columns=cols)
results = pd.DataFrame()

# define the nucleus channel and parameters
chindex = 0
sigma_spot_detection = 5
sigma_outline = 1
minsize = 100  # minimum object size [pixel]
maxsize = 500  # maximum object size [pixel]

In [31]:
# define region properties to be measured and their units
to_measure = ('label',
              'area',
              'centroid',
              'max_intensity',
              'mean_intensity',
              'min_intensity',
              'bbox'
              )

units = ["micron**2", "pixel", "pixel", "cts", "counts", "cts", ]

In [32]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(filepath)

# check if dimensions are None (because they do not exist for that image)
size_c = misc_tools.check_dimsize(mdata.image.SizeC, set2value=1)
size_z = misc_tools.check_dimsize(mdata.image.SizeZ, set2value=1)
size_t = misc_tools.check_dimsize(mdata.image.SizeT, set2value=1)
size_s = misc_tools.check_dimsize(mdata.image.SizeS, set2value=1)

In [33]:
# open the original CZI document to read 2D image planes
with pyczi.open_czi(filepath) as czidoc_r:

    # read 2d array by looping over the planes except for the channel
    for s, t, z in product(range(size_s),
                           range(size_t),
                           range(size_z)):

        # get the current plane indices and store them
        values = {'S': s, 'T': t, 'Z': z, 'C': chindex, 'Number': 0}

        # read 2D plane in case there are (no) scenes
        if mdata.image.SizeS is None:
            image2d = czidoc_r.read(plane={'T': t, 'Z': z, 'C': chindex})[..., 0]
        else:
            image2d = czidoc_r.read(plane={'T': t, 'Z': z, 'C': chindex}, scene=s)[..., 0]

        # do the voroni-otsu segmentation with GPU accelleration
        labels = cleseg_voroni_otsu(image2d,
                                    sigma_spot_detection=sigma_spot_detection,
                                    sigma_outline=sigma_outline,
                                    convert2numpy=True,
                                    verbose=False)

        # clear the border by removing "touching" objects
        labels = segmentation.clear_border(labels)

        # measure the specified parameters store in dataframe
        props = pd.DataFrame(measure.regionprops_table(labels,
                                                       intensity_image=image2d,
                                                       properties=to_measure)).set_index('label')

        # filter objects by size
        props = props[(props['area'] >= minsize) & (props['area'] <= maxsize)]

        # add well information from CZI metadata
        try:
            props['WellId'] = mdata.sample.well_array_names[s]
            props['Well_ColId'] = mdata.sample.well_colID[s]
            props['Well_RowId'] = mdata.sample.well_rowID[s]
        except (IndexError, KeyError) as error:
            print('Error:', error)
            print('Well Information not found. Using S-Index.')
            props['WellId'] = s
            props['Well_ColId'] = s
            props['Well_RowId'] = s

        # add plane indices
        props['S'] = s
        props['T'] = t
        props['Z'] = z
        props['C'] = chindex

        values = {"WellId": props['WellId'],
                  "Well_ColId": props['Well_ColId'],
                  "Well_RowId": props['Well_RowId'],
                  "S": s,
                  "T": t,
                  "Z": z,
                  "C": chindex,
                  "Number": props.shape[0]}

        print('Well:', props['WellId'].iloc[0], ' Objects: ', values['Number'])

        # update dataframe containing the number of objects
        objects = pd.concat([objects, pd.DataFrame(values, index=[0])], ignore_index=True)
        results = pd.concat([results, props], ignore_index=True)

  0%|          | 0/2 [00:00<?, ?it/s]

Well: A1  Objects:  107
Well: A2  Objects:  102


In [34]:
# reorder dataframe with single objects and show some results
new_order = list(results.columns[-7:]) + list(results.columns[:-7])
results = results.reindex(columns=new_order)
results[:5]

,WellId,Well_ColId,Well_RowId,S,T,Z,C,area,centroid-0,centroid-1,max_intensity,mean_intensity,min_intensity,bbox-0,bbox-1,bbox-2,bbox-3
0,A1,1,1,0,0,0,0,218.0,66.527523,15.954128,1295.0,1091.706422,964.0,54,11,80,23
1,A1,1,1,0,0,0,0,431.0,1121.257541,16.011601,3247.0,1988.218097,830.0,1108,7,1137,28
2,A1,1,1,0,0,0,0,360.0,149.441667,29.652778,5566.0,3049.311111,843.0,136,21,164,39
3,A1,1,1,0,0,0,0,313.0,266.402556,51.632588,1567.0,1131.194888,964.0,256,39,279,64
4,A1,1,1,0,0,0,0,471.0,714.566879,63.908705,7089.0,3380.713376,731.0,704,49,728,78
